In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1b4.dev15
cdv, version 1.1.2
Python 3.10.7


In [2]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G2Element)

from chia.consensus.default_constants import DEFAULT_CONSTANTS
from chia.types.blockchain_format.coin import Coin
from chia.types.blockchain_format.program import Program
from chia.types.coin_spend import CoinSpend
from chia.types.condition_opcodes import ConditionOpcode
from chia.types.spend_bundle import SpendBundle
from chia.util.hash import std_hash
from chia.wallet.puzzles import p2_delegated_puzzle_or_hidden_puzzle

from clvm.casts import int_to_bytes
from clvm_tools.clvmc import compile_clvm_text
from clvm_tools.binutils import disassemble

# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../code")
from utils import (load_program, print_program, print_puzzle, print_json, print_push_tx_result)
import singleton_utils
import tic_tac_toe

# puzzles
tic_tac_toe_puzzle = load_program("../code/tic-tac-toe.clsp", ["../code"])
coin_puzzle = load_program("../code/coin.clsp", ["../code"])
terminate_puzzle = load_program("../code/terminate-game.clsp", ["../code"])

waiting_room_puzzle = load_program("../code/waiting-room.clsp", ["../code"])
play_amount = 1_000_000_000_000
game_amount = (play_amount * 2) + 1 # singleton odd mojos

In [3]:
from cdv.test import Network, Wallet

network: Network = await Network.create()

# use as function parameters
get_coin_records_by_parent_ids = network.sim_client.get_coin_records_by_parent_ids
get_coin_records_by_puzzle_hash = network.sim_client.get_coin_records_by_puzzle_hash
get_coin_record_by_name = network.sim_client.get_coin_record_by_name
get_puzzle_and_solution = network.sim_client.get_puzzle_and_solution        
get_block_records = network.sim_client.get_block_records
get_additions_and_removals = network.sim_client.get_additions_and_removals

await network.farm_block()

alice: Wallet = network.make_wallet("alice")
bob: Wallet = network.make_wallet("bob")
await network.farm_block(farmer=alice)
await network.farm_block(farmer=bob)

print(f'alice balance:\t\t{alice.balance()}')
print(f'alice pk:\t\t{alice.pk()}')
print(f'alice puzzle hash:\t{alice.puzzle_hash}')
print(f'bob balance:\t\t{bob.balance()}')
print(f'alice pk:\t\t{alice.pk()}')
print(f'bob puzzle hash:\t{bob.puzzle_hash}')

# prepare players info
player_one_info = Program.to([alice.pk(), alice.puzzle_hash])
player_two_info = Program.to([bob.pk(), bob.puzzle_hash])

alice_sk = alice.sk_
alice_pk = alice.pk()
alice_puzzle_hash = alice.puzzle_hash
bob_sk = bob.sk_
bob_pk = bob.pk()
bob_puzzle_hash = bob.puzzle_hash

alice balance:		2000000000000
alice pk:		aba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8
alice puzzle hash:	4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3
bob balance:		2000000000000
alice pk:		aba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8
bob puzzle hash:	87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5


In [4]:
from chia.util.bech32m import encode_puzzle_hash
## alice
alice_waiting_room_puzzle = waiting_room_puzzle.curry(
    alice_puzzle_hash,
    alice_pk, 
    bob_pk,
    0, # P1_COIN_ID is null
    play_amount,
    game_amount
)
print_program(alice_waiting_room_puzzle)

alice_waiting_room_puzzle_hash = alice_waiting_room_puzzle.get_tree_hash()
print(f'\npuzzle hash: {alice_waiting_room_puzzle_hash.hex()}')
alice_waiting_room_address = encode_puzzle_hash(alice_waiting_room_puzzle_hash, "txch")
print(f'address: {alice_waiting_room_address}')

(a (q 2 (q 2 (i 383 (q 2 30 (c 2 (c 11 (c 23 (c 47 (c -65 (c 383 ()))))))) (q 4 (c 20 (c 28 ())) (c (c 18 (c 5 (c 95 ()))) (c (c 16 (c (a (i 47 (q . 23) (q . 11)) 1) (c (sha256 5 95) ()))) ())))) 1) (c (q ((50 . 61) 82 . 100) (51 . 60) 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 2 (i 23 (q 4 (c 24 (c (sha256 23 95) ())) ()) (q 4 (c 18 (c 22 (c 47 ()))) (c (c 24 (c 95 ())) (c (c 26 (c 95 ())) (c (c 16 (c 5 (c 95 ()))) ()))))) 1) 1)) (c (q . 0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3) (c (q . 0xaba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8) (c (q . 0xb01da35fe872b5c9c342d2a3ac6dfbe800eaf35c687a512a172d329f3bd10a83e675018a90a36178c81c0687b49d167f) (c (q) (c (q . 0x00e8d4a51000) (c (q . 0x01d1a94a2001) 1)))))))

puzzle hash: 4fef5d120ed637afed882f830fa800b97c3b6d5cb31acff9ec68f95dd51f7653
address: txch1flh46ysw6cm6lmvg97psl2qqh97rkm2ukvdvl70vdru4m4glwefsh7yyt6


In [5]:
# alice's coin creates a waiting room coin
alice_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
alice_conditions = [
    [
        ConditionOpcode.CREATE_COIN,
        alice_waiting_room_puzzle_hash,
        play_amount + 1
    ],
    
    # change
    [
        ConditionOpcode.CREATE_COIN, 
        alice.puzzle_hash, 
        alice_coin_wrapper.coin.amount - (play_amount + 1)
    ]
]

coin_spend, signature = alice_coin_wrapper.create_standard_spend(alice_sk, alice_conditions)
print_json(coin_spend.to_json_dict())
print(signature)

{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
        "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0a042c855d234578415254b7870b711fb25e8f85beaa4a66bd0673d394c761fa156406c2e3bb375d5b18766d2a12cc918ff018080",
    "solution": "0xff80ffff01ffff33ffa04fef5d120ed637afed882f830fa800b97c3b6d5cb31acff9ec68f95dd51f7653ff8600e8d4a5100180ffff33ffa04f45877796d7a6

In [6]:
from chia.util.bech32m import encode_puzzle_hash
## alice
alice_waiting_room_puzzle = waiting_room_puzzle.curry(
    alice_puzzle_hash,
    alice_pk, 
    bob_pk,
    0, # P1_COIN_ID is null
    play_amount,
    game_amount
)
alice_waiting_room_puzzle_hash = alice_waiting_room_puzzle.get_tree_hash()

In [7]:
# alice's coin creates a waiting room coin
alice_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
alice_conditions = [
    [
        ConditionOpcode.CREATE_COIN,
        alice_waiting_room_puzzle_hash,
        play_amount + 1
    ],
    
    # change
    [
        ConditionOpcode.CREATE_COIN, 
        alice.puzzle_hash, 
        alice_coin_wrapper.coin.amount - (play_amount + 1)
    ]
]

coin_spend, signature = alice_coin_wrapper.create_standard_spend(alice_sk, alice_conditions)
print_json(coin_spend.to_json_dict())

{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
        "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0a042c855d234578415254b7870b711fb25e8f85beaa4a66bd0673d394c761fa156406c2e3bb375d5b18766d2a12cc918ff018080",
    "solution": "0xff80ffff01ffff33ffa04fef5d120ed637afed882f830fa800b97c3b6d5cb31acff9ec68f95dd51f7653ff8600e8d4a5100180ffff33ffa04f45877796d7a6

In [8]:
from chia.types.spend_bundle import SpendBundle
spend_bundle = SpendBundle(
    [ coin_spend],
    signature
)
result = await network.push_tx(spend_bundle)
print_push_tx_result(result)

additions:
Coin { parent_coin_info: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba, puzzle_hash: 4fef5d120ed637afed882f830fa800b97c3b6d5cb31acff9ec68f95dd51f7653, amount: 1000000000001 }
Coin { parent_coin_info: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba, puzzle_hash: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3, amount: 749999999999 }
removals:
Coin { parent_coin_info: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001, puzzle_hash: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3, amount: 1750000000000 }


In [19]:
coin_records = await get_coin_records_by_puzzle_hash(alice_waiting_room_puzzle_hash)
alice_waiting_room_coin = coin_records[0].coin
print(coin_records[0])

{'coin': {'amount': 1000000000001,
          'parent_coin_info': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba',
          'puzzle_hash': '0x4fef5d120ed637afed882f830fa800b97c3b6d5cb31acff9ec68f95dd51f7653'},
 'coinbase': False,
 'confirmed_block_index': 3,
 'spent_block_index': 0,
 'timestamp': 1}


## Clawback

In [10]:
from utils import print_json
from chia.types.coin_spend import CoinSpend

# empty launcher_announcement for clawback spend
solution = Program.to([
    0
])

alice_waiting_room_coin_spend = CoinSpend(
        alice_waiting_room_coin,
        alice_waiting_room_puzzle,
        solution
)
print_json(alice_waiting_room_coin_spend.to_json_dict())

{
    "coin": {
        "amount": 1000000000001,
        "parent_coin_info": "0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba",
        "puzzle_hash": "0x4fef5d120ed637afed882f830fa800b97c3b6d5cb31acff9ec68f95dd51f7653"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff82017fffff01ff02ff1effff04ff02ffff04ff0bffff04ff17ffff04ff2fffff04ff81bfffff04ff82017fff8080808080808080ffff01ff04ffff04ff14ffff04ff1cff808080ffff04ffff04ff12ffff04ff05ffff04ff5fff80808080ffff04ffff04ff10ffff04ffff02ffff03ff2fffff0117ffff010b80ff0180ffff04ffff0bff05ff5f80ff80808080ff8080808080ff0180ffff04ffff01ffffff323dff5264ffff333cffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ff02ffff03ff17ffff01ff04ffff04ff18ffff04ffff0bff17ff5f80ff808080ff8080ffff01ff04ffff04ff12ffff04ff16ffff04ff2fff80808080ffff04ffff04ff18ffff04ff5fff808080ffff04ffff04ff1affff04ff5fff808080ffff04ffff04ff10ffff04ff05ffff04ff5fff80808080ff808080808080ff0180ff018080ffff04ffff01a04f45877796d7a64e19

In [11]:
from chia.types.spend_bundle import SpendBundle
from chia.util.hash import std_hash
from clvm.casts import int_to_bytes

message: bytes = std_hash(alice_puzzle_hash + int_to_bytes(play_amount))
sig_alice_spend: G2Element = AugSchemeMPL.sign(
    alice_sk,
    message
    + alice_waiting_room_coin.name()
    + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
)
spend_bundle = SpendBundle([alice_waiting_room_coin_spend],
    sig_alice_spend
)
print_json(spend_bundle.to_json_dict(include_legacy_keys = False, exclude_modern_keys = False))

{
    "aggregated_signature": "0x8d66b771632c513d54c02cdfd134940738825633d605b302b4421493531621267fc1d4eafba0382d37952e00d03bb8890aceda6d80bc1cb7c19259194fcafb277da20c7d56c91923cff97b85f97626b2dc827dc36ad22f0923eb9fd0b8e34f20",
    "coin_spends": [
        {
            "coin": {
                "amount": 1000000000001,
                "parent_coin_info": "0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba",
                "puzzle_hash": "0x4fef5d120ed637afed882f830fa800b97c3b6d5cb31acff9ec68f95dd51f7653"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff82017fffff01ff02ff1effff04ff02ffff04ff0bffff04ff17ffff04ff2fffff04ff81bfffff04ff82017fff8080808080808080ffff01ff04ffff04ff14ffff04ff1cff808080ffff04ffff04ff12ffff04ff05ffff04ff5fff80808080ffff04ffff04ff10ffff04ffff02ffff03ff2fffff0117ffff010b80ff0180ffff04ffff0bff05ff5f80ff80808080ff8080808080ff0180ffff04ffff01ffffff323dff5264ffff333cffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64

In [13]:
result = await network.push_tx(spend_bundle)
result

{'error': 'Err.ASSERT_HEIGHT_RELATIVE_FAILED'}

In [17]:
network.sim.pass_blocks(100)
network.sim.get_height()

203

In [21]:
result = await network.push_tx(spend_bundle)
result

{'error': 'Err.ASSERT_HEIGHT_RELATIVE_FAILED'}